In [ ]:
import requests
import time
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import mean, median, stdev
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

class MOAAPIPerformanceTester:
    def __init__(self, api_url):
        self.api_url = api_url
        self.results = []

    def single_request_test(self):
        """執行單次請求測試"""
        start_time = time.time()
        try:
            response = requests.get(self.api_url, timeout=30)
            end_time = time.time()

            result = {
                'timestamp': datetime.now(),
                'response_time': end_time - start_time,
                'status_code': response.status_code,
                'content_length': len(response.content),
                'success': response.status_code == 200,
                'headers': dict(response.headers)
            }

            if response.status_code == 200:
                try:
                    # 嘗試解析JSON內容
                    data = response.json()
                    result['data_records'] = len(data) if isinstance(data, list) else 1
                    result['content_type'] = 'json'
                except:
                    result['data_records'] = None
                    result['content_type'] = 'other'
            else:
                result['data_records'] = None
                result['content_type'] = None

        except requests.exceptions.Timeout:
            result = {
                'timestamp': datetime.now(),
                'response_time': 30.0,
                'status_code': 'TIMEOUT',
                'content_length': 0,
                'success': False,
                'data_records': None,
                'content_type': None,
                'headers': {}
            }
        except Exception as e:
            result = {
                'timestamp': datetime.now(),
                'response_time': None,
                'status_code': f'ERROR: {str(e)}',
                'content_length': 0,
                'success': False,
                'data_records': None,
                'content_type': None,
                'headers': {}
            }

        return result

    def multiple_requests_test(self, num_requests=10, delay=1):
        """執行多次請求測試"""
        print(f"開始執行 {num_requests} 次請求測試...")
        self.results = []

        for i in range(num_requests):
            print(f"執行第 {i+1}/{num_requests} 次請求", end="")
            result = self.single_request_test()
            self.results.append(result)

            if result['success']:
                print(f" ✓ ({result['response_time']:.2f}s)")
            else:
                print(f" ✗ ({result['status_code']})")

            if i < num_requests - 1:  # 最後一次不需要延遲
                time.sleep(delay)

        print("測試完成！")
        return self.results

    def analyze_results(self):
        """分析測試結果"""
        if not self.results:
            print("沒有測試結果可以分析")
            return None

        successful_results = [r for r in self.results if r['success']]
        response_times = [r['response_time'] for r in successful_results if r['response_time'] is not None]

        analysis = {
            'total_requests': len(self.results),
            'successful_requests': len(successful_results),
            'failed_requests': len(self.results) - len(successful_results),
            'success_rate': len(successful_results) / len(self.results) * 100,
            'response_times': {
                'mean': mean(response_times) if response_times else None,
                'median': median(response_times) if response_times else None,
                'min': min(response_times) if response_times else None,
                'max': max(response_times) if response_times else None,
                'std': stdev(response_times) if len(response_times) > 1 else None
            },
            'content_info': {}
        }

        if successful_results:
            content_lengths = [r['content_length'] for r in successful_results]
            data_records = [r['data_records'] for r in successful_results if r['data_records'] is not None]

            analysis['content_info'] = {
                'avg_content_length': mean(content_lengths),
                'avg_data_records': mean(data_records) if data_records else None,
                'content_type': successful_results[0]['content_type']
            }

        return analysis

    def print_analysis(self):
        """列印分析結果"""
        analysis = self.analyze_results()
        if not analysis:
            return

        print("\n" + "="*50)
        print("API 效能測試結果分析")
        print("="*50)

        print(f"總請求次數: {analysis['total_requests']}")
        print(f"成功請求: {analysis['successful_requests']}")
        print(f"失敗請求: {analysis['failed_requests']}")
        print(f"成功率: {analysis['success_rate']:.1f}%")

        if analysis['response_times']['mean']:
            print(f"\n響應時間統計:")
            print(f"  平均: {analysis['response_times']['mean']:.3f} 秒")
            print(f"  中位數: {analysis['response_times']['median']:.3f} 秒")
            print(f"  最小值: {analysis['response_times']['min']:.3f} 秒")
            print(f"  最大值: {analysis['response_times']['max']:.3f} 秒")
            if analysis['response_times']['std']:
                print(f"  標準差: {analysis['response_times']['std']:.3f} 秒")

        if analysis['content_info']:
            print(f"\n內容資訊:")
            print(f"  平均內容大小: {analysis['content_info']['avg_content_length']:,.0f} bytes")
            if analysis['content_info']['avg_data_records']:
                print(f"  平均資料筆數: {analysis['content_info']['avg_data_records']:.0f}")
            print(f"  內容類型: {analysis['content_info']['content_type']}")

    def plot_results(self):
        """繪製結果圖表"""
        if not self.results:
            print("沒有結果可以繪製")
            return

        # 創建數據框
        df = pd.DataFrame(self.results)
        successful_df = df[df['success'] == True].copy()

        if successful_df.empty:
            print("沒有成功的請求可以繪製圖表")
            return

        # 設置圖表樣式
        plt.style.use('seaborn-v0_8')
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))

        # 1. 響應時間趨勢圖
        axes[0, 0].plot(range(len(successful_df)), successful_df['response_time'], 'b-o', markersize=4)
        axes[0, 0].set_title('響應時間趨勢')
        axes[0, 0].set_xlabel('請求序號')
        axes[0, 0].set_ylabel('響應時間 (秒)')
        axes[0, 0].grid(True, alpha=0.3)

        # 2. 響應時間分佈直方圖
        axes[0, 1].hist(successful_df['response_time'], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
        axes[0, 1].set_title('響應時間分佈')
        axes[0, 1].set_xlabel('響應時間 (秒)')
        axes[0, 1].set_ylabel('頻次')
        axes[0, 1].grid(True, alpha=0.3)

        # 3. 成功率餅圖
        success_counts = df['success'].value_counts()
        # 動態生成標籤，根據實際的成功/失敗情況
        pie_labels = []
        pie_colors = []
        for success_status in success_counts.index:
            if success_status:
                pie_labels.append('成功')
                pie_colors.append('lightgreen')
            else:
                pie_labels.append('失敗')
                pie_colors.append('lightcoral')

        axes[1, 0].pie(success_counts.values, labels=pie_labels, autopct='%1.1f%%',
                      colors=pie_colors)
        axes[1, 0].set_title('請求成功率')

        # 4. 內容大小趨勢
        axes[1, 1].plot(range(len(successful_df)), successful_df['content_length'], 'g-s', markersize=4)
        axes[1, 1].set_title('回應內容大小趨勢')
        axes[1, 1].set_xlabel('請求序號')
        axes[1, 1].set_ylabel('內容大小 (bytes)')
        axes[1, 1].grid(True, alpha=0.3)

        plt.tight_layout()
        # 增加圖表間距以容納下方說明文字
        plt.subplots_adjust(bottom=0.15)
        plt.show()

    def export_results_to_csv(self, filename='moa_api_test_results.csv'):
        """匯出結果到CSV檔案"""
        if not self.results:
            print("沒有結果可以匯出")
            return

        df = pd.DataFrame(self.results)
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"結果已匯出到 {filename}")

# 主要執行程式碼
def main():
    # API URL
    api_url = "https://data.moa.gov.tw/Service/OpenData/FromM/FarmTransData.aspx"

    # 創建測試器
    tester = MOAAPIPerformanceTester(api_url)

    print("MOA 農委會 API 效能測試工具")
    print("="*40)
    print(f"測試目標: {api_url}")
    print()

    # 執行單次測試
    print("1. 執行單次請求測試...")
    single_result = tester.single_request_test()
    if single_result['success']:
        print(f"✓ 單次測試成功 - 響應時間: {single_result['response_time']:.3f} 秒")
        print(f"  狀態碼: {single_result['status_code']}")
        print(f"  內容大小: {single_result['content_length']:,} bytes")
        if single_result['data_records']:
            print(f"  資料筆數: {single_result['data_records']}")
    else:
        print(f"✗ 單次測試失敗 - {single_result['status_code']}")
        return

    print()

    # 執行多次測試
    print("2. 執行多次請求測試...")
    num_tests = 10  # 可以調整測試次數
    delay = 2       # 請求間隔秒數

    results = tester.multiple_requests_test(num_requests=num_tests, delay=delay)

    # 分析結果
    tester.print_analysis()

    # 繪製圖表
    print("\n3. 繪製效能圖表...")
    tester.plot_results()

    # 匯出結果
    print("\n4. 匯出測試結果...")
    tester.export_results_to_csv()

    print("\n測試完成！")

# 在 Colab 中執行
if __name__ == "__main__":
    main()